## 8. Robustness and Performance

### 67 Use `datetime` Instead of `time` for Local Clocks

In [1]:
import logging

In [2]:
import time

In [3]:
time_format = '%Y-%m-%d %H:%M:%S'

In [4]:
now = 1615347462
local_tuple = time.localtime(now)
time_str = time.strftime(time_format, local_tuple)
print(time_str)

time_tuple = time.strptime(time_str, time_format)
utc_now = time.mktime(time_tuple)
print(utc_now)

2021-03-10 12:37:42
1615347462.0


In [5]:
import os

if os.name == 'nt':
    print("This example doesn't work on Windows")
else:
    parse_format = '%Y-%m-%d %H:%M:%S %Z'
    depart_icn = '2021-03-10 12:37:42 KST'
    time_tuple = time.strptime(depart_icn, parse_format)
    time_str = time.strftime(time_format, time_tuple)
    print(time_str)

2021-03-10 12:37:42


In [6]:
try:
    arrival_nyc = '2021-03-11 02:37:42 EDT'
    time_tuple = time.strptime(arrival_nyc, time_format)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-0e86ab40354f>", line 3, in <module>
    time_tuple = time.strptime(arrival_nyc, time_format)
  File "/usr/local/lib/python3.8/_strptime.py", line 562, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "/usr/local/lib/python3.8/_strptime.py", line 352, in _strptime
    raise ValueError("unconverted data remains: %s" %
ValueError: unconverted data remains:  EDT


In [7]:
from datetime import datetime, timezone

In [8]:
now = datetime(2021, 3, 10, 3, 37, 42)      # 시간대가 아니 설정된 시간
now_utc = now.replace(tzinfo=timezone.utc)  # 시간대를 UTC로 강제 지정
now_local = now_utc.astimezone()            # UTC 시간을 디폴트 시간대로 변환
print(now_local)

2021-03-10 12:37:42+09:00


In [9]:
time_str = '2021-03-10 12:37:42'
now = datetime.strptime(time_str, time_format)  # 시간대가 아니 설정된 시간 문자열에서
time_tuple = now.timetuple()                    # 유닉스 시간 구조체로 변환
utc_now = time.mktime(time_tuple)               # 유닉스 타임스탬프 생성
print(utc_now)

1615347462.0


In [10]:
import pytz

In [11]:
arrival_nyc = '2021-03-11 02:37:42'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)  # 시간대가 아니 설정된 시간
eastern = pytz.timezone('US/Eastern')                       # 동부 시간대
nyc_dt = eastern.localize(nyc_dt_naive)                     # 동부 시간대로 시간대를 변경
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))    # UTC로 변경
print(utc_dt)

2021-03-11 07:37:42+00:00


In [12]:
kr_tz = pytz.timezone('Asia/Seoul')                # 한국 시간대
kr_dt = kr_tz.normalize(utc_dt.astimezone(kr_tz))  # 한국 시간대로 시간대를 변경
print(kr_dt)

2021-03-11 16:37:42+09:00


In [13]:
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2021-03-11 13:22:42+05:45


> - 여러 시간대를 변환할 때는 `time` 모듈을 쓰지 말라.
> - 여러 다른 시간대를 신뢰할 수 있게 변환하고 싶으면 `datetime`과 `pytz` 모듈을 함께 사용하라.
> - 항상 시간을 UTC로 표시하고, 최종적으로 표현하기 직전에 지역 시간으로 변환하라.